In [130]:
import cobra.test
import os
from os.path import join
from cobra.io import read_sbml_model
model = cobra.io.load_json_model(join("RehMBEL1391_sbml_L3V1.json"))
from cobra import Model, Reaction, Metabolite

# Adding D-glucose to the medium

The model we are using has D-fructorse as carbon source.

## Standard medium content of the model

In [131]:
model.medium   #EX_fru_e is D-frucose

{'EX_fe2_e': 10.0,
 'EX_mg2_e': 10.0,
 'EX_pi_e': 100.0,
 'EX_cobalt2_e': 10.0,
 'EX_cl_e': 10.0,
 'EX_k_e': 10.0,
 'EX_fe3_e': 10.0,
 'EX_so4_e': 10.0,
 'EX_fru_e': 3.0,
 'EX_nh4_e': 10.0,
 'EX_na_e': 10.0,
 'EX_o2_e': 18.5,
 'EX_mobd_e': 10.0,
 'EX_h2o_e': 1000.0,
 'EX_h_e': 100.0}

We will be trying to include D-glucose in the medium and look at the bimass and production of PHB.

With the current medium the biomass and yeald of phb

In [133]:
with model:
    model.objective = model.reactions.EX_pbhb_e
    solution= model.optimize()
    print('The current theoretical maximum biomass productivity:', solution.fluxes['EX_BIOMASS_c'], 'h')
    print('----------------')
    phb_production=model.slim_optimize()
    print('The current maximum theoretical productivity for making PHB:', phb_production , 'mmol/gDW*h')
    print('----------------')
    maximum_yield = phb_production / (-1*(model.reactions.EX_fru_e.flux))
    print('Maximum theoretical yield =', maximum_yield, 'mmol phb/mmol fructose') 

0.0
The current theoretical maximum biomass productivity: <Solution 3.763 at 0x2ade154f8e0> h
----------------
The current maximum theoretical productivity for making PHB: 3.76271186440678 mmol/gDW*h
----------------
Maximum theoretical yield = 1.2542372881355932 mmol phb/mmol fructose


### To add D-glucose to the medium we need new reactions to make R.eutropha take up and use D-glucose as a carbon source. 

In [30]:
reaction = Reaction('D-GLC')
reaction.name = 'D-glucose transport via PEP:Pyr PTS'
reaction.subsystem = 'Cell Envelope Biosynthesis'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default

In [31]:
EX_glc_D_e = Metabolite(
    'EX_glc_D_e',
    formula='C6H12O6',
    name='D-glucose',
    compartment='c')

In [32]:
reaction.add_metabolites({
    EX_glc_D_e: -1.0,
    model.metabolites.pep_c: -1.0,
    model.metabolites.pyr_c: 1.0,
    model.metabolites.g6p_c: 1.0
})
reaction.reaction  # This gives a string representation of the reaction

'EX_glc_D_e + pep_c --> g6p_c + pyr_c'

In [84]:
with model:
    medium = model.medium
    medium['EX_fru_e'] = 0
    medium['EX_succ_e'] = 10
    print(medium)
    model.medium=medium
    model.objective = model.reactions.EX_pbhb_e
    print(model.slim_optimize())
    

{'EX_fe2_e': 10.0, 'EX_mg2_e': 10.0, 'EX_pi_e': 100.0, 'EX_cobalt2_e': 10.0, 'EX_cl_e': 10.0, 'EX_k_e': 10.0, 'EX_fe3_e': 10.0, 'EX_so4_e': 10.0, 'EX_fru_e': 0, 'EX_nh4_e': 10.0, 'EX_na_e': 10.0, 'EX_o2_e': 18.5, 'EX_mobd_e': 10.0, 'EX_h2o_e': 1000.0, 'EX_h_e': 100.0, 'EX_succ_e': 10}
6.454545454545457


In [108]:
#model.reactions.get_by_id('EX_succ_e')

In [106]:
#for metabolites in model.metabolites.query('succ','name'):
#    print(metabolites.name)

In [107]:
#model.reactions.query('succ','name')

In [85]:
with model:
    medium = model.medium
    medium['EX_fru_e'] = 0
    #medium['EX_glc_D_e'] = 0
    medium['EX_succ_e'] = 0
    model.medium =medium
    print(medium)
    print(model.slim_optimize())

{'EX_fe2_e': 10.0, 'EX_mg2_e': 10.0, 'EX_pi_e': 100.0, 'EX_cobalt2_e': 10.0, 'EX_cl_e': 10.0, 'EX_k_e': 10.0, 'EX_fe3_e': 10.0, 'EX_so4_e': 10.0, 'EX_fru_e': 0, 'EX_nh4_e': 10.0, 'EX_na_e': 10.0, 'EX_o2_e': 18.5, 'EX_mobd_e': 10.0, 'EX_h2o_e': 1000.0, 'EX_h_e': 100.0, 'EX_succ_e': 0}
0.0


In [101]:
with model:
    medium = model.medium
    medium['EX_fru_e'] = 10
    medium['EX_o2_e']=0
    model.medium=medium
    print(model.slim_optimize(), model.objective)

5.454545454545443 Maximize
1.0*EX_pbhb_e - 1.0*EX_pbhb_e_reverse_f6e04


In [105]:
with model:
    medium = model.medium
    medium['EX_fru_e'] = 10
    medium['EX_o2_e'] =100
    model.medium=medium
    
    
    Objective=model.objective
    solution=model.optimize()
    print(Objective)
    print('The current theoretical maximum biomass productivity:', solution.fluxes['Biomass'], 'h')
    print('----------------')
    
    model.objective = model.reactions.EX_pbhb_e
    print(model.objective)
    phb_production=model.slim_optimize()
    print('The current maximum theoretical productivity for making PHB:', phb_production , 'mmol/gDW*h')
    print('----------------')
    
    maximum_yield = phb_production / (-1*(model.reactions.EX_fru_e.flux))
    print('Maximum theoretical yield =', maximum_yield, 'mmol phb/mmol fructose')
    model.objective = model.reactions.EX_pbhb_e
    phb_production=model.slim_optimize()
    print(phb_production)

Maximize
1.0*EX_pbhb_e - 1.0*EX_pbhb_e_reverse_f6e04
The current theoretical maximum biomass productivity: 0.0 h
----------------
Maximize
1.0*EX_pbhb_e - 1.0*EX_pbhb_e_reverse_f6e04
The current maximum theoretical productivity for making PHB: 12.542372881355949 mmol/gDW*h
----------------
Maximum theoretical yield = 1.2542372881355948 mmol phb/mmol fructose
12.542372881355949


In [96]:
    model.objective = model.reactions.EX_pbhb_e
    print(model.objective)
    phb_production=model.slim_optimize()
    print('The current maximum theoretical productivity for making PHB:', phb_production , 'mmol/gDW*h')

Maximize
1.0*EX_pbhb_e - 1.0*EX_pbhb_e_reverse_f6e04
The current maximum theoretical productivity for making PHB: 3.762711864406772 mmol/gDW*h {'EX_fe2_e': 10.0, 'EX_mg2_e': 10.0, 'EX_pi_e': 100.0, 'EX_cobalt2_e': 10.0, 'EX_cl_e': 10.0, 'EX_k_e': 10.0, 'EX_fe3_e': 10.0, 'EX_so4_e': 10.0, 'EX_fru_e': 10, 'EX_nh4_e': 10.0, 'EX_na_e': 10.0, 'EX_o2_e': 0, 'EX_mobd_e': 10.0, 'EX_h2o_e': 1000.0, 'EX_h_e': 100.0}


In [95]:
    model.objective = model.reactions.EX_pbhb_e
    phb_production=model.slim_optimize()
    print(phb_production)

3.762711864406772


In [ ]:
with model:
    medium = model.medium
    #medium['EX_fru_e'] = 10
    medium['EX_o2_e'] =0
    print(medium)
    model.medium=medium
    model.objective = model.reactions.EX_pbhb_e
    solution=model.optimize()
    print(solution.fluxes[solution.fluxes != 0.])
    print(solution)

In [67]:
for reactions in model.metabolites.get_by_id('phb_c').reactions:
    print(reactions)

PHAS: r3hbcoa_c <=> coa_c + phb_c
PHBt: h_c + phb_c --> h_e + phb_e
PHBD: h2o_c + phb_c <=> h_c + r3hbn2_c + r3rhbb_c


In [ ]:
###########See if it makes more of phb